In [42]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import sklearn as sk
import dtale
import re, datetime
from helper_functions import *
%config Completer.use_jedi = True
%load_ext autoreload
%autoreload 2

In [1]:
# df = pd.read_csv("donors.csv", low_memory=False).drop(columns="Unnamed: 0")
# df.to_pickle("donors.pkl")
# sample = df.iloc[:1000, :].copy()
# sample.to_pickle("sample.pkl")

C:\Users\vasco\anaconda3\envs\DM\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
#df = pd.read_pickle("donors.pkl")
#sample = pd.read_pickle("sample.pkl")

# Variables

In [3]:
# Categorical variables
cat_vars = [
    "STATE", # category
    
    "NOEXCH", # binary
    "RECINHSE", # binary
    "RECP3", # binary
    "RECPGVG", # binary
    "RECSWEEP", # binary
    
    "MAJOR", # binary
    "MDMAUD", # category
    "MDMAUD_R", # category
    "MDMAUD_F", # category
    "MDMAUD_A", # category
    
    "DOMAIN", # category
    "HOMEOWNR", # binary
    "CHILD03", # category
    "CHILD07", # category
    "CHILD12", # category
]

# Continuous variables
cont_vars = [
    "AGE", # 
    "INCOME", #
]

# Variables for plots
plot_vars = [
    "STATE", 
    "ZIP", # clean zip [TODO]
    "GEOCODE2", 
    "ODATEDW", # turn into year-month-01 [TODO]
]

# not used features
drop_vars = [
    "GEOCODE", # 80k nan
    "MAILCODE", 
    "OSOURCE", 
    "DATASRCE",
    "PVASTATE",
    "LIFESRC",
]

# features to process
fe_features = [
    "TCODE", # Shorten it [TODO]
    "DOB", # Create age groups [DONE]
    "ODATEDW" # Create loyalty groups
]

# feature engineering
cat_fe_features = [
    "ageGroup", # category
    "loyaltyGroup", # category
]

cont_fe_features = [
    
]

# EDA DONE

In [4]:
# Converting date columns into datetimes
for c in (dateNames := [c for c in df.columns.values if ("DATE" in c)]) :
    df[c] = pd.to_datetime(df[c], infer_datetime_format=True)

In [5]:
# MAIL OFFERS
# The following variables indicate the number of
# known times the donor has responded to other
# types of mail order offers.

groupMailOffers = df.loc[:, "MBCRAFT":"PUBOPP"].columns.to_list()

# dtale.show(df[mailOrderOffers])

for col in groupMailOffers:
    df[col] = df[col].fillna(0).astype(np.int16)

In [6]:
searchName = "MDMAUD"
namesToGroup = [col for col in df.columns.to_list() if searchName in col]
print(f"{namesToGroup=}")

for name in namesToGroup:
    setattr(df, name, df[name].str.replace("X", "").replace("", np.nan))

# MAJOR
namesToGroup = ["MAJOR"] + namesToGroup
df.MAJOR.replace({"X": True, " ": False}, inplace=True)

# RFA_2
groupRFA_2 = ["RFA_2", "RFA_2R", "RFA_2F", "RFA_2A"]
namesToGroup += groupRFA_2
df[namesToGroup].head(2)

namesToGroup=['MDMAUD', 'MDMAUD_R', 'MDMAUD_F', 'MDMAUD_A']


,MAJOR,MDMAUD,MDMAUD_R,MDMAUD_F,MDMAUD_A,RFA_2,RFA_2R,RFA_2F,RFA_2A
0,False,NaN,NaN,NaN,NaN,L4E,L,4,E
1,False,NaN,NaN,NaN,NaN,L2G,L,2,G


In [7]:
## TODO: Create categorical orders

incomeCats = ["Low", "Medium", "High"]
incomeBins = [0, 3, 6, 10]

df["cat_INCOME"] = pd.cut(
    df.INCOME, bins=incomeBins, labels=incomeCats
)

df.drop(columns=["INCOME"], inplace=True)

for c in namesToGroup:
    df[c] = pd.Categorical(df[c])

In [8]:
# ADATE: Date the promotion was mailed
# RDATE: Date the gift was received for promotion
# RFA: RFA status as of promotion date
# RAMNT: Dollar amount of the gift for the promotion
# RFA_R: Recency
    # F: First time, 1 donation 0-6 months ago => 12 months
    # N: New, 1 donation 7-12 months ago or more than one donation 0-6 months ago => 12 months
    # A: Active, first donation more than 12 months ago and donated in the last 12 months => 12 months
    # L: Lapsing, last donation between 13-24 months ago => 13-24 months
    # I: Inactive donor, donation 25+ months ago => 25-36 months
    # S: Star, 3 CONSECUTIVE card mailings
# RFA_F: Frequency
    # 1: One gift in recency period
    # 2: Two gifts in recency period
    # 3: Three gifts in recency period
    # 4: Four or more gifts in recency period
# RFA_A: Amount
    # A=$0.01  -  $1.99
    # B=$2.00  -  $2.99
    # C=$3.00  -  $4.99
    # D=$5.00  -  $9.99
    # E=$10.00 - $14.99
    # F=$15.00 - $24.99
    # G=$25.00 and above


toReorder = ["ADATE", "RFA", "RDATE", "RAMNT"]
reorderedCols = []
for i in range(3, 25):
    reorderedCols += [str(col) + "_" + str(i) for col in toReorder]
reorderedCols = ["ADATE_2"] + groupRFA_2 + reorderedCols

dictIdx = {col: df.columns.get_loc(col) for col in reorderedCols}

temp = df[dictIdx.keys()].copy()
df.drop(columns=dictIdx.keys(), inplace=True)

for key in dictIdx.keys():
    df[key] = temp[key]

In [9]:
max(df.LASTDATE)

Timestamp('2017-02-01 00:00:00')

In [10]:
# AGE RELATED
df.DOB = pd.to_datetime(df.DOB)
df["AGE"] = np.int16(2020 - pd.to_datetime(df.DOB).dt.year)
df.AGE = df.AGE.where(df.AGE != 0, np.nan)

# age_groups_labels = ["0-18", "18-30", "30-60", "60+"]
age_groups_labels = ["Minor", "Young worker", "Worker", "Retired"]
age_groups_values = [0, 18, 30, 60, np.inf]

df["ageGroup"] = pd.cut(df.AGE, age_groups_values, labels=age_groups_labels)

In [11]:
# LOYALTY GROUP
# years passed since first gift
df["loyaltyAge"] = np.int16(2020 - pd.to_datetime(df.ODATEDW).dt.year)
# Origin groups [loyalty measured in years]
origin_groups_labels = ["0-1", "1-2", "2-5", "5-10", "10-20", "20+"]
origin_groups_values = [0, 1, 2, 5, 10, 20, np.inf]

df["loyaltyGroup"] = pd.cut(
    df.loyaltyAge, bins=origin_groups_values, labels=origin_groups_labels
)

In [12]:
condition = df.GENDER.isin(["U", "M", "F"])
df.GENDER.where(condition, "U", inplace=True)

NameError: name 'sample' is not defined

In [13]:
# Always keep this cell last before EDA WIP
sample.to_pickle("sample_eda_done.pkl")
df.to_pickle("df_eda_done.pkl")

# EDA WIP

In [14]:
# restart work from here
sample = pd.read_pickle("sample_eda_done.pkl")
df = pd.read_pickle("df_eda_done.pkl")

In [15]:
df["dateOfDelivery"] = pd.to_datetime(datetime.date(2021, 1, 4))

In [16]:
round((df["dateOfDelivery"] - df["LASTDATE"]) / np.timedelta64(1, "M"))

0        61.0
1        61.0
2        61.0
3        61.0
4        60.0
         ... 
95407    59.0
95408    58.0
95409    51.0
95410    48.0
95411    49.0
Length: 95412, dtype: float64

In [17]:
df.loc[:5, "RFA_2":"RAMNT_10"].T

,0,1,2,3,4,5
RFA_2,L4E,L2G,L4E,L4E,L2F,L1F
RFA_2R,L,L,L,L,L,L
RFA_2F,4,2,4,4,2,1
RFA_2A,E,G,E,E,F,F
ADATE_3,2016-06-01 00:00:00,2016-06-01 00:00:00,2016-06-01 00:00:00,2016-06-01 00:00:00,2016-06-01 00:00:00,2016-06-01 00:00:00
RFA_3,S4E,A2G,S4E,S4E,A2F,A1F
RDATE_3,NaT,NaT,NaT,NaT,NaT,NaT
RAMNT_3,NaN,NaN,NaN,NaN,NaN,NaN
ADATE_4,2016-04-01 00:00:00,2016-04-01 00:00:00,2016-04-01 00:00:00,2016-04-01 00:00:00,2016-04-01 00:00:00,2016-04-01 00:00:00
RFA_4,S4E,A2G,S4E,S4E,A2F,A1F


In [18]:
# Drop entries where
# df[df.ODATEDW < df.DOB]

In [19]:
tcodes="""
    000    = _
    001    = MR.
    001001 = MESSRS.
    001002 = MR. & MRS.
    002    = MRS.
    002002 = MESDAMES
    003    = MISS
    003003 = MISSES
    004    = DR.
    004002 = DR. & MRS.
    004004 = DOCTORS
    005    = MADAME
    006    = SERGEANT
    009    = RABBI
    010    = PROFESSOR
    010002 = PROFESSOR & MRS.
    010010 = PROFESSORS
    011    = ADMIRAL
    011002 = ADMIRAL & MRS.
    012    = GENERAL
    012002 = GENERAL & MRS.
    013    = COLONEL
    013002 = COLONEL & MRS.
    014    = CAPTAIN
    014002 = CAPTAIN & MRS.
    015    = COMMANDER
    015002 = COMMANDER & MRS.
    016    = DEAN
    017    = JUDGE
    017002 = JUDGE & MRS.
    018    = MAJOR
    018002 = MAJOR & MRS.
    019    = SENATOR
    020    = GOVERNOR
    021002 = SERGEANT & MRS.
    022002 = COLNEL & MRS.
    024    = LIEUTENANT
    026    = MONSIGNOR
    027    = REVEREND
    028    = MS.
    028028 = MSS.
    029    = BISHOP
    031    = AMBASSADOR
    031002 = AMBASSADOR & MRS.
    033    = CANTOR
    036    = BROTHER
    037    = SIR
    038    = COMMODORE
    040    = FATHER
    042    = SISTER
    043    = PRESIDENT
    044    = MASTER
    046    = MOTHER
    047    = CHAPLAIN
    048    = CORPORAL
    050    = ELDER
    056    = MAYOR
    059002 = LIEUTENANT & MRS.
    062    = LORD
    063    = CARDINAL
    064    = FRIEND
    065    = FRIENDS
    068    = ARCHDEACON
    069    = CANON
    070    = BISHOP
    072002 = REVEREND & MRS.
    073    = PASTOR
    075    = ARCHBISHOP
    085    = SPECIALIST
    087    = PRIVATE
    089    = SEAMAN
    090    = AIRMAN
    091    = JUSTICE
    092    = MR. JUSTICE
    100    = M.
    103    = MLLE.
    104    = CHANCELLOR
    106    = REPRESENTATIVE
    107    = SECRETARY
    108    = LT. GOVERNOR
    109    = LIC.
    111    = SA.
    114    = DA.
    116    = SR.
    117    = SRA.
    118    = SRTA.
    120    = YOUR MAJESTY
    122    = HIS HIGHNESS
    123    = HER HIGHNESS
    124    = COUNT
    125    = LADY
    126    = PRINCE
    127    = PRINCESS
    128    = CHIEF
    129    = BARON
    130    = SHEIK
    131    = PRINCE AND PRINCESS
    132    = YOUR IMPERIAL MAJEST
    135    = M. ET MME.
    210    = PROF.
"""

# print(
#     re.sub(
#         r'0*(?P<keepleft>\d+)\W*=\W*(?P<keep>.*)', 
#         r'"\g<keepleft>" : "\g<keep>",', 
#         tcodes
#     ).strip()
# )

In [20]:
dtypes = df.dtypes.to_dict()
types_dict = {
    "TCODE": str,
}
dtypes.update(types_dict)

dictTCODE = {
    "0" : "_",
    "1" : "MR.",
    "1001" : "MESSRS.",
    "1002" : "MR. & MRS.",
    "2" : "MRS.",
    "2002" : "MESDAMES",
    "3" : "MISS",
    "3003" : "MISSES",
    "4" : "DR.",
    "4002" : "DR. & MRS.",
    "4004" : "DOCTORS",
    "5" : "MADAME",
    "6" : "SERGEANT",
    "9" : "RABBI",
    "10" : "PROFESSOR",
    "10002" : "PROFESSOR & MRS.",
    "10010" : "PROFESSORS",
    "11" : "ADMIRAL",
    "11002" : "ADMIRAL & MRS.",
    "12" : "GENERAL",
    "12002" : "GENERAL & MRS.",
    "13" : "COLONEL",
    "13002" : "COLONEL & MRS.",
    "14" : "CAPTAIN",
    "14002" : "CAPTAIN & MRS.",
    "15" : "COMMANDER",
    "15002" : "COMMANDER & MRS.",
    "16" : "DEAN",
    "17" : "JUDGE",
    "17002" : "JUDGE & MRS.",
    "18" : "MAJOR",
    "18002" : "MAJOR & MRS.",
    "19" : "SENATOR",
    "20" : "GOVERNOR",
    "21002" : "SERGEANT & MRS.",
    "22002" : "COLNEL & MRS.",
    "24" : "LIEUTENANT",
    "26" : "MONSIGNOR",
    "27" : "REVEREND",
    "28" : "MS.",
    "28028" : "MSS.",
    "29" : "BISHOP",
    "31" : "AMBASSADOR",
    "31002" : "AMBASSADOR & MRS.",
    "33" : "CANTOR",
    "36" : "BROTHER",
    "37" : "SIR",
    "38" : "COMMODORE",
    "40" : "FATHER",
    "42" : "SISTER",
    "43" : "PRESIDENT",
    "44" : "MASTER",
    "46" : "MOTHER",
    "47" : "CHAPLAIN",
    "48" : "CORPORAL",
    "50" : "ELDER",
    "56" : "MAYOR",
    "59002" : "LIEUTENANT & MRS.",
    "62" : "LORD",
    "63" : "CARDINAL",
    "64" : "FRIEND",
    "65" : "FRIENDS",
    "68" : "ARCHDEACON",
    "69" : "CANON",
    "70" : "BISHOP",
    "72002" : "REVEREND & MRS.",
    "73" : "PASTOR",
    "75" : "ARCHBISHOP",
    "85" : "SPECIALIST",
    "87" : "PRIVATE",
    "89" : "SEAMAN",
    "90" : "AIRMAN",
    "91" : "JUSTICE",
    "92" : "MR. JUSTICE",
    "100" : "M.",
    "103" : "MLLE.",
    "104" : "CHANCELLOR",
    "106" : "REPRESENTATIVE",
    "107" : "SECRETARY",
    "108" : "LT. GOVERNOR",
    "109" : "LIC.",
    "111" : "SA.",
    "114" : "DA.",
    "116" : "SR.",
    "117" : "SRA.",
    "118" : "SRTA.",
    "120" : "YOUR MAJESTY",
    "122" : "HIS HIGHNESS",
    "123" : "HER HIGHNESS",
    "124" : "COUNT",
    "125" : "LADY",
    "126" : "PRINCE",
    "127" : "PRINCESS",
    "128" : "CHIEF",
    "129" : "BARON",
    "130" : "SHEIK",
    "131" : "PRINCE AND PRINCESS",
    "132" : "YOUR IMPERIAL MAJEST",
    "135" : "M. ET MME.",
    "210" : "PROF.",
}

In [21]:
# Categories of TCODE
df["cat_TCODE"] = df.TCODE.replace(to_replace=dictTCODE)
condition = df.cat_TCODE.isin(dictTCODE.values())
df.cat_TCODE.where(condition, "_", inplace=True)
# Here we assume that _ means unknown, not that they do not have a title
df.cat_TCODE.replace("_", np.nan)

0         NaN
1         MR.
2         MR.
3         NaN
4         NaN
         ... 
95407     MR.
95408     MR.
95409     MR.
95410     NaN
95411    MRS.
Name: cat_TCODE, Length: 95412, dtype: object

In [22]:
df.AVGGIFT.mean()

13.347785980803197

In [70]:
df.groupby(by="cat_TCODE", axis=0)["AGE"].median()

cat_TCODE
BROTHER             NaN
CAPTAIN            75.0
CAPTAIN & MRS.     60.0
CHAPLAIN           71.0
COLNEL & MRS.       NaN
COLONEL            77.0
COLONEL & MRS.     77.0
COMMODORE          68.0
DOCTORS            37.0
DR.                60.0
DR. & MRS.         61.0
ELDER              70.0
FATHER              NaN
GENERAL             NaN
JUDGE              76.0
LIEUTENANT         63.0
M.                 10.0
MAJOR              64.0
MAJOR & MRS.       76.5
MASTER              NaN
MISS               60.5
MR.                64.0
MR. & MRS.         60.0
MRS.               70.0
MS.                59.0
MSS.               74.0
RABBI              40.0
REVEREND           81.0
REVEREND & MRS.     NaN
SERGEANT           75.0
SISTER             80.0
SR.                69.0
_                  60.0
Name: AGE, dtype: float64

In [23]:
df.columns.tolist()

['ODATEDW',
 'OSOURCE',
 'TCODE',
 'STATE',
 'ZIP',
 'MAILCODE',
 'PVASTATE',
 'DOB',
 'NOEXCH',
 'RECINHSE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'MDMAUD',
 'DOMAIN',
 'HOMEOWNR',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'NUMCHLD',
 'GENDER',
 'WEALTH1',
 'HIT',
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
 'DATASRCE',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'SOLP3',
 'SOLIH',
 'MAJOR',
 'WEALTH2',
 'GEOCODE',
 'COLLECT1',
 'VETERANS',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'PETS',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES',
 'LIFESRC',
 'PEPSTRFL',
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'ETH7',
 'ETH8',
 'ETH9',
 '

In [28]:
features_census = [
    "POP901",
    "POP902",
    "POP903",
    "POP90C1",
    "POP90C2",
    "POP90C3",
    "POP90C4",
    "POP90C5",
    "ETH1",
    "ETH2",
    "ETH3",
    "ETH4",
    "ETH5",
    "ETH6",
    "ETH7",
    "ETH8",
    "ETH9",
    "ETH10",
    "ETH11",
    "ETH12",
    "ETH13",
    "ETH14",
    "ETH15",
    "ETH16",
    "AGE901",
    "AGE902",
    "AGE903",
    "AGE904",
    "AGE905",
    "AGE906",
    "AGE907",
    "CHIL1",
    "CHIL2",
    "CHIL3",
    "AGEC1",
    "AGEC2",
    "AGEC3",
    "AGEC4",
    "AGEC5",
    "AGEC6",
    "AGEC7",
    "CHILC1",
    "CHILC2",
    "CHILC3",
    "CHILC4",
    "CHILC5",
    "HHAGE1",
    "HHAGE2",
    "HHAGE3",
    "HHN1",
    "HHN2",
    "HHN3",
    "HHN4",
    "HHN5",
    "HHN6",
    "MARR1",
    "MARR2",
    "MARR3",
    "MARR4",
    "HHP1",
    "HHP2",
    "DW1",
    "DW2",
    "DW3",
    "DW4",
    "DW5",
    "DW6",
    "DW7",
    "DW8",
    "DW9",
    "HV1",
    "HV2",
    "HV3",
    "HV4",
    "HU1",
    "HU2",
    "HU3",
    "HU4",
    "HU5",
    "HHD1",
    "HHD2",
    "HHD3",
    "HHD4",
    "HHD5",
    "HHD6",
    "HHD7",
    "HHD8",
    "HHD9",
    "HHD10",
    "HHD11",
    "HHD12",
    "ETHC1",
    "ETHC2",
    "ETHC3",
    "ETHC4",
    "ETHC5",
    "ETHC6",
    "HVP1",
    "HVP2",
    "HVP3",
    "HVP4",
    "HVP5",
    "HVP6",
    "HUR1",
    "HUR2",
    "RHP1",
    "RHP2",
    "RHP3",
    "RHP4",
    "HUPA1",
    "HUPA2",
    "HUPA3",
    "HUPA4",
    "HUPA5",
    "HUPA6",
    "HUPA7",
    "RP1",
    "RP2",
    "RP3",
    "RP4",
    "MSA",
    "ADI",
    "DMA",
    "IC1",
    "IC2",
    "IC3",
    "IC4",
    "IC5",
    "IC6",
    "IC7",
    "IC8",
    "IC9",
    "IC10",
    "IC11",
    "IC12",
    "IC13",
    "IC14",
    "IC15",
    "IC16",
    "IC17",
    "IC18",
    "IC19",
    "IC20",
    "IC21",
    "IC22",
    "IC23",
    "HHAS1",
    "HHAS2",
    "HHAS3",
    "HHAS4",
    "MC1",
    "MC2",
    "MC3",
    "TPE1",
    "TPE2",
    "TPE3",
    "TPE4",
    "TPE5",
    "TPE6",
    "TPE7",
    "TPE8",
    "TPE9",
    "PEC1",
    "PEC2",
    "TPE10",
    "TPE11",
    "TPE12",
    "TPE13",
    "LFC1",
    "LFC2",
    "LFC3",
    "LFC4",
    "LFC5",
    "LFC6",
    "LFC7",
    "LFC8",
    "LFC9",
    "LFC10",
    "OCC1",
    "OCC2",
    "OCC3",
    "OCC4",
    "OCC5",
    "OCC6",
    "OCC7",
    "OCC8",
    "OCC9",
    "OCC10",
    "OCC11",
    "OCC12",
    "OCC13",
    "EIC1",
    "EIC2",
    "EIC3",
    "EIC4",
    "EIC5",
    "EIC6",
    "EIC7",
    "EIC8",
    "EIC9",
    "EIC10",
    "EIC11",
    "EIC12",
    "EIC13",
    "EIC14",
    "EIC15",
    "EIC16",
    "OEDC1",
    "OEDC2",
    "OEDC3",
    "OEDC4",
    "OEDC5",
    "OEDC6",
    "OEDC7",
    "EC1",
    "EC2",
    "EC3",
    "EC4",
    "EC5",
    "EC6",
    "EC7",
    "EC8",
    "SEC1",
    "SEC2",
    "SEC3",
    "SEC4",
    "SEC5",
    "AFC1",
    "AFC2",
    "AFC3",
    "AFC4",
    "AFC5",
    "AFC6",
    "VC1",
    "VC2",
    "VC3",
    "VC4",
    "ANC1",
    "ANC2",
    "ANC3",
    "ANC4",
    "ANC5",
    "ANC6",
    "ANC7",
    "ANC8",
    "ANC9",
    "ANC10",
    "ANC11",
    "ANC12",
    "ANC13",
    "ANC14",
    "ANC15",
    "POBC1",
    "POBC2",
    "LSC1",
    "LSC2",
    "LSC3",
    "LSC4",
    "VOC1",
    "VOC2",
    "VOC3",
    "HC1",
    "HC2",
    "HC3",
    "HC4",
    "HC5",
    "HC6",
    "HC7",
    "HC8",
    "HC9",
    "HC10",
    "HC11",
    "HC12",
    "HC13",
    "HC14",
    "HC15",
    "HC16",
    "HC17",
    "HC18",
    "HC19",
    "HC20",
    "HC21",
    "MHUC1",
    "MHUC2",
    "AC1",
    "AC2",
]

In [31]:
features = df.columns.tolist()
features_non_census = features
for f in features_census:
    features_non_census.remove(f)
features_non_census

['ODATEDW',
 'OSOURCE',
 'TCODE',
 'STATE',
 'ZIP',
 'MAILCODE',
 'PVASTATE',
 'DOB',
 'NOEXCH',
 'RECINHSE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'MDMAUD',
 'DOMAIN',
 'HOMEOWNR',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'NUMCHLD',
 'GENDER',
 'WEALTH1',
 'HIT',
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
 'DATASRCE',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'SOLP3',
 'SOLIH',
 'MAJOR',
 'WEALTH2',
 'GEOCODE',
 'COLLECT1',
 'VETERANS',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'PETS',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES',
 'LIFESRC',
 'PEPSTRFL',
 'CARDPROM',
 'MAXADATE',
 'NUMPROM',
 'CARDPM12',
 'NUMPRM12',
 'RAMNTALL',
 'NGIFTALL',
 'CARDGIFT',
 'MINRAMNT',
 'MINRDATE',
 'MAXRAMNT',
 'MAXRDATE',
 'LASTGIFT',
 'LASTDA

In [40]:
df[features_non_census]

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,ADATE_24,RFA_24,RDATE_24,RAMNT_24,AGE,ageGroup,loyaltyAge,loyaltyGroup,dateOfDelivery,cat_TCODE
0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,,...,2014-06-01,S4E,2014-06-01,9.0,63.0,Retired,11,10-20,2021-01-04,_
1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,,...,2014-06-01,F1E,NaT,NaN,48.0,Worker,6,5-10,2021-01-04,MR.
2,2010-01-01,AMH,1,NC,27017,,,NaT,0,,...,2014-06-01,S3D,2014-06-01,6.0,NaN,NaN,10,5-10,2021-01-04,MR.
3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,,...,NaT,,NaT,NaN,72.0,Retired,13,10-20,2021-01-04,_
4,2006-01-01,,0,FL,33176,,,1940-01-01,0,X,...,2014-06-01,A3D,NaT,NaN,80.0,Retired,14,10-20,2021-01-04,_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95407,2016-01-01,ASE,1,AK,99504,,,NaT,0,,...,NaT,,NaT,NaN,NaN,NaN,4,2-5,2021-01-04,MR.
95408,2016-01-01,DCD,1,TX,77379,,,1970-01-01,0,,...,NaT,,NaT,NaN,50.0,Worker,4,2-5,2021-01-04,MR.
95409,2015-01-01,MBC,1,MI,48910,,,1958-01-01,0,,...,NaT,,NaT,NaN,62.0,Retired,5,2-5,2021-01-04,MR.
95410,2006-01-01,PRV,0,CA,91320,,,1960-05-01,0,X,...,2014-06-01,S3F,NaT,NaN,60.0,Worker,14,10-20,2021-01-04,_


In [ ]:
# Flags

flagsBinary = [
    "NOEXCH",  # binary
    "RECINHSE",  # binary
    "RECP3",  # binary
    "RECPGVG",  # binary
    "RECSWEEP",  # binary
    "MAJOR",  # binary
]

df[flagsBinary].replace({"X": True, " ": False, "0": False, "1": True})

In [ ]:
# Family Aggregate Data - loc[:, "DOMAIN":"WEALTH1"]
# FAD_name

domainCats = [
    letter + str(i) for i in range(1, 4) for letter in ["U", "C", "S", "T", "R"]
]

domainCats.reverse()
domainCats.insert(4, "U4")

df["cat_FAD_domain"] = pd.Categorical(df.DOMAIN, categories=domainCats, ordered=True)